# Código para raspagem de dados do Banco Nacional de Medidas Penais, CNJ
O código vai raspar, de forma automática, informações que não são disponibilizadas pela planilha fornecida pelo Banco Nacional de Medidas Penas (BNMP), mantido pelo Conselho Nacional de Justiça. 

A ideia é ampliar as possibilidades de análise e traçar panoramas da situação de seguridade pública do país.

## Preparando o ambiente
Para a raspagem dos dados, utilizaremos as bibliotecas Selenium. Para a análise, utilizaremos o Pandas. 
Já para a representação dos dados utilizaremos o matplotlib e Seaborn. 

In [1]:

#Biblioteca para raspagem 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

#Bibliotecas para várias funções relacionadas à tempo
import time
from datetime import datetime

#Bibliotecas para análise dos dados
import pandas as pd 
import numpy as np 
import requests as requests
import matplotlib.pyplot as plt 
import seaborn as sns


# Inicialização do raspador

In [2]:
driver = webdriver.Chrome()
driver.get('https://portalbnmp.cnj.jus.br/#/captcha/%2Fpesquisa-peca')
WebDriverWait(driver, 10)

<selenium.webdriver.support.wait.WebDriverWait (session="5c31091dbe11408578535c9f8979142d")>

In [ ]:
#WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']")))
#WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()

In [3]:
driver.maximize_window() # For maximizing window
driver.implicitly_wait(5) # gives an implicit wait for 20 seconds

# Teste do primeiro item

In [4]:
searchbox_npeca = driver.find_element(By.XPATH, "//input[@name='numeroProcesso']").send_keys('0002717-37.2024.8.26.0521.01.0001-13')
driver.implicitly_wait(5)
search_button = driver.find_element(By. XPATH, '/html/body/app-root/div/div/div[2]/div/app-pesquisa-peca/div[1]/p-fieldset/fieldset/div/div/form/div[18]/button[2]/span[1]')
search_button.click()

driver.implicitly_wait(5)

    
processo_click = driver.find_element(By. XPATH, '/html/body/app-root/div/div/div[2]/div/app-pesquisa-peca/div[1]/p-datatable/div/div[1]/table/tbody/tr/td[1]/span[2]/span').click()

driver.implicitly_wait(10)

print (driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/div[4]/table/tbody/tr/td').text)
print (driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/p-datatable/div/div[1]/table/thead/tr/th/span').text)
print (driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/p-datatable/div/div[1]/table/tbody/tr/td/span').text)
print ("Pena imposta:", driver.find_element(By.XPATH, '//*[@id="ui-panel-0-content"]/div[1]/p[18]/span').text,
       '\n', "Regime:", driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/p[19]/span').text)

ABRAÃO MENDES DE SOUSA
Tipificações penais:
10826, 16;
Pena imposta: 3 ano(s) 6 mês(es) 0 dia(s). 
 Regime: Semiaberto


# Rodagem Automatizada

In [5]:
mandados_sorocaba = pd.read_csv('C:\Downloads\matheus.arruda\Downloads\PROJETOS\DADOS\mandados_sor_10.csv', sep= ',')
mandados_sorocaba.head()

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\matheus.arruda\AppData\Local\Temp\ipykernel_15220\2562798416.py:1: SyntaxWarning: invalid escape sequence '\D'
  mandados_sorocaba = pd.read_csv('C:\Downloads\matheus.arruda\Downloads\PROJETOS\DADOS\mandados_sor_10.csv', sep= ',')


,Número,Nome,Alcunha,Nome da Mãe,Nome do Pai,Data de Nascimento,Situação,Data,Órgão Expedidor,Peça,AbertoHá
0,7001840-61.2003.8.26.0602.01.0003-23,ALEXANDRE ALVES FERREIRA,XANDE,IZABEL APARECIDA PINHAN FERREIRA,JOAO ALVES FERREIRA FILHO,05/06/1978,Pendente de Cumprimento,17/04/2013,DEPARTAMENTO DE EXECUÇÕES CRIMINAIS - 10 RAJ D...,Mandado de Prisão,11
1,7000988-78.2006.8.26.0037.01.0003-07,WLADEMIR SCHWINDEN,NAO INFORMADO,TANIA MARA SCHWINDEN,EVALDIR SCHWINDEN,09/01/1978,Pendente de Cumprimento,20/01/2011,JURI/EXECUCOES DE SOROCABA,Mandado de Prisão,13


In [6]:
#Vamos entrar no endereço novamente
driver.get('https://portalbnmp.cnj.jus.br/#/captcha/%2Fpesquisa-peca')

In [7]:
dados = []
bd_foragidos = pd.DataFrame()

for index, row in mandados_sorocaba.iterrows():
    processo = row['Número']
        
    #Primeiro, vamos passar nosso número de processo
    driver.find_element(By.XPATH, "/html/body/app-root/div/div/div[2]/div/app-pesquisa-peca/div[1]/p-fieldset/fieldset/div/div/form/div[14]/span/p-inputmask/input").send_keys(f'{processo}')
    
    driver.implicitly_wait(5)
    
    #Então, vamos clicar em PESQUISAR
    driver.find_element(By. XPATH, '/html/body/app-root/div/div/div[2]/div/app-pesquisa-peca/div[1]/p-fieldset/fieldset/div/div/form/div[18]/button[2]/span[1]').click()
   
    #driver.implicitly_wait(5)
    
    time.sleep(5)

    #E, depois, abrir o link do processo
    driver.find_element(By. XPATH, '/html/body/app-root/div/div/div[2]/div/app-pesquisa-peca/div[1]/p-datatable/div/div[1]/table/tbody/tr/td[1]/span[2]/span').click()
    
    driver.implicitly_wait(5)
    
    time.sleep(5)

    nome = driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/div[4]/table/tbody/tr/td').text
    crime = driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/p-datatable/div/div[1]/table/tbody/tr/td/span').text
#pena = driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[1]/p[19]/span').text

    print(nome, crime)

    dados.append({
    "Nome": nome,
    "Crime(s)": crime,
    })
    
    time.sleep(5)
    
    driver.find_element(By.XPATH, '/html/body/app-root/div/div/div[2]/div/app-resumo-peca/div/p-panel/div/div[2]/div[2]/p-footer/div/button[3]').click()
   
    time.sleep(5)

bd_foragidos = dados

ALEXANDRE ALVES FERREIRA 2848, 157, § 2º, II;
WLADIMIR SCHIMINDEN 2848, 121, § 2º, IV;


In [8]:
bd_foragidos = pd.DataFrame(dados)
bd_foragidos   

,Nome,Crime(s)
0,ALEXANDRE ALVES FERREIRA,"2848, 157, § 2º, II;"
1,WLADIMIR SCHIMINDEN,"2848, 121, § 2º, IV;"


In [9]:
driver.quit()